In [ ]:
import pandas as pd

ModuleNotFoundError: No module named 'tabulate'

In [ ]:
# Load the data
data = pd.read_csv("train.csv", low_memory=False)
data.drop_duplicates() # remove duplicates

,Characteristics.LotFeatures,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.features_reso.results,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,...,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.Heating,Structure.LivingArea,Structure.NewConstructionYN,Structure.ParkingFeatures,Structure.Rooms.RoomsTotal,Structure.YearBuilt,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,3200.0,NaN,NaN,NaN,NaN,NaN,"['Cooling.CeilingFans', 'Flooring.Carpet', 'Fl...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,0.0,commr,NaN
1,NaN,NaN,NaN,3.8,3.7,3.8,3.8,"['Appliances.GasRange', 'Appliances.Range', 'A...",NaN,4.2,...,NaN,NaN,['natural gas'],3175.0,False,['off alley'],6.0,NaN,NaN,NaN
2,NaN,18750.0,NaN,NaN,NaN,NaN,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,1926.0,commr,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['CommunityFeatures.Lake', 'WaterfrontFeatures...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,agric,NaN
4,"['horses allowed', 'paddock', 'pond(s)']",NaN,NaN,3.6,NaN,NaN,3.6,"['Appliances.Refrigerator', 'Appliances.Microw...",NaN,4.2,...,NaN,NaN,['other'],0.0,False,"['unassigned', 'off street']",4.0,2006.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,NaN,NaN,NaN,3.8,NaN,NaN,3.8,"['Levels.One', 'ParkingFeatures.Attached', 'Pa...",NaN,4.1,...,NaN,2.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
107433,NaN,NaN,2.8,NaN,3.6,3.1,3.1,"['Appliances.Dishwasher', 'Appliances.Stainles...",3.2,NaN,...,0.0,NaN,"['natural gas', 'forced air']",0.0,False,"['assigned', 'off street']",9.0,1963.0,NaN,NaN
107434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['ExteriorFeatures.PrivateYard', 'ExteriorFeat...",NaN,NaN,...,NaN,2.5,NaN,NaN,False,['driveway'],NaN,NaN,NaN,NaN
107435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['ParkingFeatures.ParkingLot'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# data cleaning of columnes_numeriques
# replace 2_stories by 2
# data['ImageData.style.stories.summary.label'] = data['ImageData.style.stories.summary.label'].replace('2_stories', 2)

In [ ]:
columnes_numeriques = data.select_dtypes(include=['number'])  # Selecciona columnes numèriques
columnes_categoriques = data.select_dtypes(exclude=['number'])  # Selecciona columnes no numèriques (categòriques)

In [ ]:
#move from columnes_categoriques to columnes_numeriques[ImageData.style.stories.summary.label]
columnes_numeriques['ImageData.style.stories.summary.label'] = columnes_categoriques['ImageData.style.stories.summary.label']
columnes_categoriques = columnes_categoriques.drop(columns=['ImageData.style.stories.summary.label'])

In [ ]:
columnes_numeriques.to_csv('columnes_numeriques.csv', index=False)
columnes_categoriques.to_csv('columnes_categoriques.csv', index=False)

In [ ]:
columnes_numeriques.head()

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
0,3200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,NaN,NaN,3.8,3.7,3.8,3.8,NaN,4.2,4.0,4.0,...,2.0,1.0,1.0,NaN,NaN,NaN,NaN,3175.0,6.0,NaN
2,18750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1926.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3.6,NaN,NaN,3.6,NaN,4.2,NaN,NaN,...,1.0,0.0,2.0,NaN,NaN,NaN,NaN,0.0,4.0,2006.0


In [ ]:
columnes_categoriques.head()

,Characteristics.LotFeatures,ImageData.features_reso.results,ImageData.room_type_reso.results,ImageData.style.exterior.summary.label,ImageData.style.stories.summary.label,Listing.Dates.CloseDate,Listing.ListingId,Location.Address.CensusBlock,Location.Address.CensusTract,Location.Address.City,...,Location.Area.SubdivisionName,Location.School.HighSchoolDistrict,Property.PropertyType,Structure.Basement,Structure.Cooling,Structure.Heating,Structure.NewConstructionYN,Structure.ParkingFeatures,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,"['Cooling.CeilingFans', 'Flooring.Carpet', 'Fl...","['Other', 'FrontOfStructure', 'Basement', 'Sta...",NaN,NaN,2023-12-22T00:00:00,mrd09114306,17-195-300-1004,17-195-300,morrison,...,NaN,NaN,commercial sale,NaN,['central air'],NaN,False,NaN,commr,NaN
1,NaN,"['Appliances.GasRange', 'Appliances.Range', 'A...","['SideOfStructure', 'Other', 'Kitchen', 'Office']",NaN,NaN,2023-09-19T00:00:00,mrd09478582,17-97-865400-2007,17-97-865400,highland park,...,NaN,112,residential,['partial'],['window/wall unit - 1'],['natural gas'],False,['off alley'],NaN,NaN
2,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...","['Community', 'FrontOfStructure', 'Other', 'Ga...",bungalow,2_stories,2023-12-04T00:00:00,mrd09595738,17-31-210900-2014,17-31-210900,chicago,...,NaN,NaN,commercial sale,NaN,['central air'],NaN,False,NaN,commr,NaN
3,NaN,"['CommunityFeatures.Lake', 'WaterfrontFeatures...","['Yard', 'View']",NaN,NaN,2023-08-01T00:00:00,mrd09604772,NaN,NaN,wilmington,...,NaN,209u,farm,NaN,NaN,NaN,False,NaN,agric,NaN
4,"['horses allowed', 'paddock', 'pond(s)']","['Appliances.Refrigerator', 'Appliances.Microw...","['FrontOfStructure', 'BackOfStructure', 'Commu...",ranch,1_story,2023-09-20T00:00:00,mrd09795957,17-89-852403-1029,17-89-852403,maple park,...,NaN,301,residential,['full'],"['partial', 'none']",['other'],False,"['unassigned', 'off street']",NaN,NaN


In [ ]:
columnes_numeriques = columncolumnes_numeriqueses_numeriques.apply(lambda col: col.fillna(col.mean(skipna=True)) if col.dtype in ['float64', 'int64'] else col)
columnes_numeriques

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
0,3200.000000,3.116429,3.308723,3.211074,3.117166,3.182633,3.311895,3.499023,3.108145,3.20957,...,1.921669,0.441973,3.102167,701.495539,649.350608,0.941737,2.039585,1742.211448,7.174756,0.000000
1,59491.450533,3.116429,3.800000,3.700000,3.800000,3.800000,3.311895,4.200000,4.000000,4.00000,...,2.000000,1.000000,1.000000,701.495539,649.350608,0.941737,2.039585,3175.000000,6.000000,1969.155199
2,18750.000000,3.116429,3.308723,3.211074,3.117166,3.182633,3.311895,3.499023,3.108145,3.20957,...,1.921669,0.441973,3.102167,701.495539,649.350608,0.941737,2.039585,1742.211448,7.174756,1926.000000
3,59491.450533,3.116429,3.308723,3.211074,3.117166,3.182633,3.311895,3.499023,3.108145,3.20957,...,1.921669,0.441973,3.102167,701.495539,649.350608,0.941737,2.039585,1742.211448,7.174756,1969.155199
4,59491.450533,3.116429,3.600000,3.211074,3.117166,3.600000,3.311895,4.200000,3.108145,3.20957,...,1.000000,0.000000,2.000000,701.495539,649.350608,0.941737,2.039585,0.000000,4.000000,2006.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,59491.450533,3.116429,3.800000,3.211074,3.117166,3.800000,3.311895,4.100000,3.108145,3.20957,...,1.000000,0.000000,2.000000,701.495539,649.350608,0.941737,2.000000,1742.211448,7.174756,1969.155199
107433,59491.450533,2.800000,3.308723,3.600000,3.100000,3.100000,3.200000,3.499023,3.100000,2.70000,...,3.000000,0.000000,4.000000,701.495539,649.350608,0.000000,2.039585,0.000000,9.000000,1963.000000
107434,59491.450533,3.116429,3.308723,3.211074,3.117166,3.182633,3.311895,3.499023,3.108145,3.20957,...,3.000000,1.000000,5.000000,701.495539,649.350608,0.941737,2.500000,1742.211448,7.174756,1969.155199
107435,59491.450533,3.116429,3.308723,3.211074,3.117166,3.182633,3.311895,3.499023,3.108145,3.20957,...,1.921669,0.441973,3.102167,701.495539,649.350608,0.941737,2.039585,1742.211448,7.174756,1969.155199
